# sandbox for testing multi-thread programming

In [3]:
import concurrent.futures
import urllib.request

URLS = ['http://www.foxnews.com/',
        'http://www.cnn.com/',
        'http://europe.wsj.com/',
        'http://www.bbc.co.uk/',
        'http://nonexistent-subdomain.python.org/']

# Retrieve a single page and report the URL and contents
def load_url(url, timeout):
    with urllib.request.urlopen(url, timeout=timeout) as conn:
        return conn.read()

# We can use a with statement to ensure threads are cleaned up promptly
with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
    # Start the load operations and mark each future with its URL
    future_to_url = {executor.submit(load_url, url, 60): url for url in URLS}
    for future in concurrent.futures.as_completed(future_to_url):
        url = future_to_url[future]
        try:
            data = future.result()
        except Exception as exc:
            print('%r generated an exception: %s' % (url, exc))
        else:
            print('%r page is %d bytes' % (url, len(data)))

'http://nonexistent-subdomain.python.org/' generated an exception: <urlopen error [Errno 11001] getaddrinfo failed>
'http://europe.wsj.com/' generated an exception: HTTP Error 403: Forbidden
'http://www.bbc.co.uk/' generated an exception: <urlopen error [Errno 11001] getaddrinfo failed>
'http://www.foxnews.com/' generated an exception: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>
'http://www.cnn.com/' generated an exception: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>


# testing Generators in python

In [3]:
def my_generator():
    yield 1
    yield 2
    yield 3

gen = my_generator()
print(next(gen))
print(next(gen))

1
2


# testing pickle lib in python

In [12]:
import pickle

def fib():
    a, b = 0, 1
    while 1:
       yield b
       a, b = b, a+b

a = fib()

# Pickling (serializing) the object to a file
with open('fib.pkl', 'wb') as f:
    for i in range(500_000):
        pickle.dump(next(a), f)

# Unpickling (deserializing) the object from the file
with open('fib.pkl', 'rb') as f:
    loaded_data = pickle.load(f)
    print("Object unpickled successfully.")

#print(f"Loaded data: {loaded_data}")

Object unpickled successfully.


In [ ]:
import struct

def fib():
    a, b = 0, 1
    while True:
        yield b
        a, b = b, a + b

N = 500_000  # number of iterations

# Write Fibonacci numbers to file in a streaming fashion
with open('fib.bin', 'wb', buffering=1024*1024) as f:  # 1MB buffer
    for i, val in zip(range(N), fib()):
        # Store as variable-length binary (length + bytes)
        data = val.to_bytes((val.bit_length() + 7) // 8 or 1, 'big')
        f.write(struct.pack('>I', len(data)))  # 4-byte length prefix
        f.write(data)


First value read: 1


In [18]:
# Read back the first number (streaming read)
with open('fib.bin', 'rb') as f:
    length = struct.unpack('>I', f.read(4))[0]
    first_val = int.from_bytes(f.read(length), 'big')
    print("First value read:", first_val)

First value read: 1


# testing various functions in python

In [1]:
num = 25  
print(bin(num), oct(num), hex(num))

0b11001 0o31 0x19


In [2]:
text = "12345abc"
print(text.startswith(("1", "12", "123")))

True


In [7]:
import itertools as iter

nums = [1, 2, 3]
# results = list(iter.permutations(nums, 2))
results = list(iter.combinations(nums, 2))
print(results)

[(1, 2), (1, 3), (2, 3)]


In [3]:
a = [2, 10, 15, 14, 8, 9, 3, 17, 16]
list(map(lambda x: 'big' if x>=10 else 'small', a))

['small', 'big', 'big', 'big', 'small', 'small', 'small', 'big', 'big']

In [4]:
a = [2, 10, 15, 14, 8, 9, 3, 17, 16]
list(filter(lambda x: x%2 == 0, a))

[2, 10, 14, 8, 16]

In [5]:
mylist = [2,3,5,8,11,14,17,102,44]
print(list(map(lambda x:'Yes' if  x%2==1 else 'No',mylist)))

['No', 'Yes', 'Yes', 'No', 'Yes', 'No', 'Yes', 'No', 'No']


In [9]:
mylist = [2,15,26,8,11,14,17,102,44]

map_list = map(lambda x:x%10,mylist)
print(map_list)

In [ ]:
def firstn(n):
    num = 0
    while (num<n):
        yield num
        num += 1

for i in firstn(10_000_000):
    print(i)